In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os

# set env flags to catch BLAS used for scipy/numpy 
# to only use 1 cpu, n_cpus will be totally controlled by csky
if False:
    os.environ['MKL_NUM_THREADS'] = "1"
    os.environ['NUMEXPR_NUM_THREADS'] = "1"
    os.environ['OMP_NUM_THREADS'] = "1"
    os.environ['OPENBLAS_NUM_THREADS'] = "1"
    os.environ['VECLIB_MAXIMUM_THREADS'] = "1"

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'w'
mpl.rcParams['savefig.facecolor'] = 'w'
import matplotlib.pyplot as plt
from matplotlib import colors, cm
import csky as cy
from csky import cext
import numpy as np
import astropy
#from icecube import astro
import histlite as hl
import healpy
import healpy as hp
import socket
import pickle
from scipy import stats
import copy
healpy.disable_warnings()
plt.rc('figure', facecolor = 'w')
plt.rc('figure', dpi=100)

## Define Settings

In [ ]:
selection_version = 'version-001-p01'

host_name = socket.gethostname()

if 'cobalt' in host_name:
    print('Working on Cobalts')
    #data_prefix = '/data/user/ssclafani/data/cscd/final'
    #ana_dir = '/data/user/ssclafani/data/analyses/'
    plot_dir = cy.utils.ensure_dir('/data/user/mhuennefeld/data/analyses/DNNCascadeCodeReview/unblinding_checks/plots/unblinding/gp_ts_energy_range')
    
else:
    raise ValueError('Unknown host:', host_name)

In [ ]:
for dir_path in [plot_dir]:
    if not os.path.exists(dir_path):
        print('Creating directory:', dir_path)
        os.makedirs(dir_path)

## Load Data

In [ ]:
repo = cy.selections.Repository()
specs = cy.selections.DNNCascadeDataSpecs.DNNC_10yr

In [ ]:
%%time

ana = cy.get_analysis(
    repo, selection_version, specs, 
    #gammas=np.r_[0.1:6.01:0.125],
)

In [ ]:
a = ana.anas[0]
a.sig

In [ ]:
a.bg_data

### Create ana with low energy only

In [ ]:
energy_cut = 10000

class DNNCascade_10yr_lowE(cy.selections.DNNCascadeDataSpecs.DNNCascade_10yr):
    def dataset_modifications(self, ds):
        print('Applying energy cut of {:3.0f} GeV'.format(energy_cut))
        
        ds.sig = ds.sig._subsample(ds.sig.energy < energy_cut)
        ds.data = ds.data._subsample(ds.data.energy < energy_cut)


In [ ]:
%%time

ana_lowE = cy.get_analysis(
    cy.selections.Repository(), selection_version, [DNNCascade_10yr_lowE], 
)

In [ ]:
a_lowE = ana_lowE.anas[0]
a_lowE.sig, a_lowE.bg_data

## Create ana with modified sigma

In [ ]:
sigma_factor = 1.3

class DNNCascade_sigma(cy.selections.DNNCascadeDataSpecs.DNNCascade_10yr):
    def dataset_modifications(self, ds):
        print('Applying sigma factor of {:3.3f}'.format(sigma_factor))
        
        ds.sig['sigma'] = ds.sig['sigma'] * sigma_factor
        ds.data['sigma'] = ds.data['sigma'] * sigma_factor


In [ ]:
%%time

ana_sigma = cy.get_analysis(
    cy.selections.Repository(), selection_version, [DNNCascade_sigma], 
)

In [ ]:
a_sigma = ana_sigma.anas[0]
a_sigma.bg_data.sigma / a.bg_data.sigma

## Helpers

In [ ]:
from cycler import cycle
from copy import deepcopy

soft_colors = cy.plotting.soft_colors
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


def get_bias_allt(tr, ntrials=200, n_sigs=np.r_[:101:10], quiet=False):
    trials = [
        (None if quiet else print(f'\r{n_sig:4d} ...', end='', flush=True))
        or
        tr.get_many_fits(ntrials, n_sig=n_sig, logging=False, seed=n_sig)
        for n_sig in n_sigs]
    if not quiet:
        print()
    for (n_sig, t) in zip(n_sigs, trials):
        t['ntrue'] = np.repeat(n_sig, len(t))
    allt = cy.utils.Arrays.concatenate(trials)
    return allt

def get_color_cycler():
    return cycle(colors)

def plot_ns_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)

    h = hl.hist((allt.ntrue, allt.ns), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(ax.set_ylim(lim))
    ax.plot(lim, lim, **expect_kw)
    ax.set_aspect('equal')

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$n_s$')
    ax.grid()

def plot_gamma_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)
    expect_gamma = tr.sig_injs[0].flux[0].gamma

    h = hl.hist((allt.ntrue, allt.gamma), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(lim)
    ax.set_ylim(1, 4)
    ax.axhline(expect_gamma, **expect_kw)

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$\gamma$')
    ax.grid()

def plot_bkg_trials(
            bg, fig=None, ax=None, 
            label='{} bg trials', 
            label_fit=r'$\chi^2[{:.2f}\mathrm{{dof}},\ \eta={:.3f}]$', 
            color=colors[0],
            density=False,
            bins=50,
        ):
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 4))
    
    if density:
        h = bg.get_hist(bins=bins).normalize(density=True)
    else:
        h = bg.get_hist(bins=bins)
    if label is not None:
        label = label.format(bg.n_total)
    hl.plot1d(ax, h, crosses=True, color=color, label=label)

    # compare with the chi2 fit:
    if hasattr(bg, 'pdf'):
        x = h.centers[0]
        norm = h.integrate().values
        if label_fit is not None:
            label_fit = label_fit.format(bg.ndof, bg.eta)
        if density:
            ax.semilogy(x, bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)
        else:
            ax.semilogy(x, norm * bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)

    ax.set_xlabel(r'TS')
    if density:
        ax.set_ylabel(r'Density')
    else:
        ax.set_ylabel(r'number of trials')
    ax.legend()
        
    return fig, ax

## Setup Analysis

In [ ]:
import sys
sys.path.insert(0, '../..')

import config as cg

cg.base_dir = '/data/user/mhuennefeld/data/analyses/unblinding_v1.0.1_csky_bugfix_template_flux/'

In [ ]:
def get_gp_tr(template_str, cutoff=np.inf, gamma=None, ana=ana, cpus=20):
    cutoff_GeV = cutoff * 1e3
    gp_conf = cg.get_gp_conf(
        template_str=template_str, gamma=gamma, 
        cutoff_GeV=cutoff_GeV, base_dir=cg.base_dir)
    
    if template_str == 'pi0' and cutoff != np.inf or ana.keys != ['DNNCascade_10yr']:
        print('Removing dir!', cutoff)
        gp_conf.pop('dir')
    
    gp_conf['extra_keeps'] = ['azimuth']

    tr = cy.get_trial_runner(gp_conf, ana=ana, mp_cpus=cpus)
    return tr

def get_template_tr(template, gamma=2.7, cutoff_tev=np.inf, cpus=20):
    cutoff_gev = cutoff_tev * 1000.
    gp_conf = {
        'template': template,
        'flux': cy.hyp.PowerLawFlux(gamma, energy_cutoff=cutoff_gev),
        'randomize': ['ra'],
        'fitter_args': dict(gamma=gamma),
        'sigsub': True,
        'update_bg': True,
        'fast_weight': False,
    }
    tr = cy.get_trial_runner(gp_conf, ana=ana, mp_cpus=cpus)
    return tr


#### Get TrialRunners

In [ ]:
tr_dict = {
    'pi0': get_gp_tr('pi0'),
    'pi0_lowE': get_gp_tr('pi0', ana=ana_lowE),
    'pi0_sigma': get_gp_tr('pi0', ana=ana_sigma),
    'kra5': get_gp_tr('kra5'),
    'kra50': get_gp_tr('kra50'),
}

#### Get bkg fits for each template

In [ ]:
bkg_file_dict = {
    'pi0': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'pi0'),
    'kra5': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'kra5'),
    'kra50': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'kra50'),
    'pi0_lowE': os.path.join(plot_dir, 'trials_pi0_lowE.pkl'),
    'pi0_sigma': os.path.join(plot_dir, 'trials_pi0_sigma.pkl'),
}
n_bkg_trials = 20000
seed = 1337

bkg_dict = {}
for key, tr in tr_dict.items():
    if key in bkg_file_dict and os.path.exists(bkg_file_dict[key]):
        print('Loading background trials for template {}'.format(key))
        sig = np.load(bkg_file_dict[key], allow_pickle=True)
        if key in ['pi0', 'kra5', 'kra50']:
            bkg_dict[key] = sig['poisson']['nsig'][0.0]['ts']
        else:
            bkg_dict[key] = sig.ts
    
    else:
        print('Running background trials for template {}'.format(key))
        trials = tr.get_many_fits(
            n_trials=n_bkg_trials, seed=seed, mp_cpus=20)
        bkg_dict[key] = trials.ts
        
        out_file = os.path.join(plot_dir, 'trials_{}.pkl'.format(key))
        with open(out_file, 'wb') as f:
            pickle.dump(trials, f, protocol=2)


#### Get Results for each template

In [ ]:
res_dict = {}
for key in tr_dict.keys():
    f_path = os.path.join(
        cg.base_dir, 
        'gp/results/{}/{}_unblinded.npy'.format(key, key), 
    )
    if os.path.exists(f_path):
        res_dict[key] = np.load(f_path)

In [ ]:
tr_dict

In [ ]:
if 'pi0_lowE' in tr_dict:
    print(tr_dict['pi0'].to_E2dNdE(671.1))
    print(tr_dict['pi0_lowE'].to_ns(tr_dict['pi0'].to_dNdE(671.1)))
    print(tr_dict['pi0_lowE'].to_E2dNdE(559.47))


In [ ]:
ns_dict = {
    'pi0': 671.1, #748.0,  # bias corected
    'pi0_lowE': 559.47, #623.58, # bias corected
    'kra5': 242.9, #275.6, # bias corected
    'kra50': 184.8, #211.1, # bias corected
    'snr': 218.6,
    'pwn': 279.6,
    'unid': 238.4,
}

#### Compute likelihood/Ratio contribution  of events

In [ ]:
from tqdm.notebook import tqdm_notebook as tqdm


def get_lr_from_trial(trial, tr, gamma=2.7):
    """Get event likelihood-ratio value of signal-subtracted likelihood
    
    Info here:
        https://wiki.icecube.wisc.edu/index.php/
        Cascade_7yr_PS_GP/Galactic_Source_Search_Methods#Signal-Subtracted_Likelihood
    """
    L = tr.get_one_llh_from_trial(trial)
    
    res = L.fit(**tr.fitter_args)
    ns = res[1]['ns']
    N = float(len(trial.evss[0][0]))
    print(res, ns, N)
    
    space_eval = cy.inspect.get_space_eval(L, -1, 0) # 0: background events (1 would be for signal events)
    energy_eval = cy.inspect.get_energy_eval(L, -1, 0)
    StoB_space_ss = space_eval(gamma=gamma)[1] 
    SoB_energy = energy_eval(gamma=gamma)[0]
    SoB_space = space_eval(gamma=gamma)[0] 
    w = (SoB_space - StoB_space_ss) * SoB_energy
    lr = w * ns/N + 1.
    return lr
    
def get_lr(template_str, gamma=2.7):
    """Get event likelihood-ratio value of signal-subtracted likelihood
    
    Info here:
        https://wiki.icecube.wisc.edu/index.php/
        Cascade_7yr_PS_GP/Galactic_Source_Search_Methods#Signal-Subtracted_Likelihood
    """
    trial = tr_dict[template_str].get_one_trial(TRUTH=True)
    return get_lr_from_trial(trial=trial, tr=tr_dict[template_str], gamma=gamma)

def compute_mc_lr(trial, tr, tr_trial, gamma=2.7):
    L = tr.get_one_llh_from_trial(trial)
    res = L.fit(**tr.fitter_args)
    ns = res[1]['ns']
    N = float(np.sum([len(t) for t in trial.evss[0]]))

    space_eval = cy.inspect.get_space_eval(L, -1, 1) # 0: background events (1 would be for signal events)
    energy_eval = cy.inspect.get_energy_eval(L, -1, 1)
    StoB_space_ss = space_eval(gamma=gamma)[1] 
    SoB_energy = energy_eval(gamma=gamma)[0]
    SoB_space = space_eval(gamma=gamma)[0] 
    w = (SoB_space - StoB_space_ss) * SoB_energy
    lr = w * ns/N + 1.

    # add true energy and append
    mc_event_i = cy.utils.Arrays(trial.evss[0][1])
    a = tr_trial.ana.anas[0]
    assert np.allclose(mc_event_i.log10energy, a.sig[trial.evss[0][1].idx].log10energy)
    mc_event_i['true_energy'] = a.sig[trial.evss[0][1].idx].true_energy
    
    return lr, mc_event_i
        
def get_mc_lr(tr, n_sig, gamma=2.7, seed=42, n_reps=100):
    """Get event likelihood-ratio value of signal-subtracted likelihood for injected MC events
    
    Info here:
        https://wiki.icecube.wisc.edu/index.php/
        Cascade_7yr_PS_GP/Galactic_Source_Search_Methods#Signal-Subtracted_Likelihood
    """
    mc_events = []
    lr_list = []
    
    for i in tqdm(range(n_reps)):
        trial = tr.get_one_trial(n_sig=n_sig, seed=seed + i)
        lr, mc_event_i = compute_mc_lr(trial=trial, tr=tr, tr_trial=tr, gamma=gamma)
        
        # append 
        mc_events.append(mc_event_i)
        lr_list.append(lr)
    
    lr_list = np.concatenate(lr_list)
    mc_events = cy.utils.Arrays.concatenate(mc_events)
    return lr_list, mc_events


In [ ]:
lr_dict = {
    'pi0': get_lr('pi0'),
    'kra5': get_lr('kra5', None),
    'kra50': get_lr('kra50', None),
}

ts_dict = {}
for key, lr in lr_dict.items():
    ts = 2 * np.log(lr)
    print(key, np.sum(ts))
    ts_dict[key] = ts
    

In [ ]:
plt.hist(ts_dict['pi0'], bins=1000)
plt.yscale('log')
pass

#### Compute MC-based LR values

In [ ]:

lr_mc_dict = {
    'pi0': get_mc_lr(tr=tr_dict['pi0'], n_sig=ns_dict['pi0']),
    'pi0_lowE': get_mc_lr(tr=tr_dict['pi0_lowE'], n_sig=ns_dict['pi0_lowE']),
    'kra5': get_mc_lr(tr=tr_dict['kra5'], n_sig=ns_dict['kra5'], gamma=None),
    'kra50': get_mc_lr(tr=tr_dict['kra50'], n_sig=ns_dict['kra50'], gamma=None),
}

ts_mc_dict = {}
for key, (lr, _) in lr_mc_dict.items():
    ts = 2 * np.log(lr)
    print(key, np.sum(ts))
    ts_mc_dict[key] = ts

#### Get event weights based on TS

In [ ]:
def get_weights_from_ts(ts, method='clip'):
    if method == 'clip':
        ts = np.clip(ts, 0., np.inf)
    elif method == 'abs':
        ts = np.abs(ts)
    elif method == 'all':
        pass
    else:
        raise ValueError('Unknown method: {}'.format(method))
    return ts / np.sum(ts)


#### Marginalized distributions

In [ ]:
def weighted_quantile(x, weights, quantile):

    if weights is None:
        weights = np.ones_like(x)

    sorted_indices = np.argsort(x)
    x_sorted = x[sorted_indices]
    weights_sorted = weights[sorted_indices]
    cum_weights = np.cumsum(weights_sorted) / np.sum(weights)
    idx = np.searchsorted(cum_weights, quantile)
    return x_sorted[idx]

def make_marginalized_plot(ts, method='clip', title=None, cols=['dec', 'ra', 'energy', 'sigma', 'azimuth'], n_bins=30, cl=0.68):
    
    weights = get_weights_from_ts(ts, method=method)
    fig, axes = plt.subplots(len(cols), 1, figsize=(9, 3*len(cols)))
    
    q_tail = (1. - cl) / 2.
    for col, ax in zip(cols, axes):
        
        # define defaults
        xlabel = col
        data = a.bg_data[col]
        bins = n_bins
        
        # modify defaults
        if col == 'dec':
            xlabel = r'Dec: $\delta_\mathrm{reco}$ / °'
            data = np.rad2deg(a.bg_data[col])
            bins = np.linspace(-90, 90., n_bins)
        elif col == 'energy':
            xlabel = r'$E_\mathrm{reco}$ / GeV'
            bins = np.logspace(np.log10(500), 7, n_bins)
        elif col == 'ra':
            xlabel = r'RA: $\alpha_\mathrm{reco}$ / °'
            data = np.rad2deg(a.bg_data[col])
            bins = np.linspace(0, 360., n_bins)
        elif col == 'sigma':
            data = np.rad2deg(a.bg_data[col])
            xlabel = r'$\sigma_\mathrm{reco}$ / °'
            bins = np.linspace(0, 45., n_bins)
        
        # define what data unit to use for quantile
        data_avg = data
        if col == 'energy':
            data_avg = a.bg_data[col]
        
        avg_q = np.average(data_avg, weights=weights)
        med_q = weighted_quantile(data_avg, weights=weights, quantile=0.5)
        lower_q = weighted_quantile(data_avg, weights=weights, quantile=q_tail)
        upper_q = weighted_quantile(data_avg, weights=weights, quantile=1. - q_tail)
        
        label = '{}: {:3.2f} | {:.0f}% CL: [{:3.2f}, {:3.2f}])'.format(
            col, avg_q, cl*100., lower_q, upper_q)
        
        ax.axvline(lower_q, ls='--', color='0.7', label='{:.0f}% CL'.format(cl*100.))
        ax.axvline(upper_q, ls='--', color='0.7')
        ax.axvspan(lower_q, upper_q, color='0.7', alpha=.2)
        ax.axvline(avg_q, ls='-.', color='0.7', label='Average')
        ax.axvline(med_q, ls='-', color='0.7', label='Median')
            
        ax.hist(data, weights=weights, bins=bins, label=label, density=True)
        ax.hist(data, histtype='step', label='All Data', bins=bins, density=True)
        ax.set_xlabel(xlabel)
        ax.set_ylabel('Density')
        if col == 'energy':
            ax.set_xscale('log')
            ax.set_yscale('log')
        ax.legend()
    
    if title is not None:
        fig.suptitle(title)
    fig.tight_layout()
    
    return fig, axes

fig, axes = make_marginalized_plot(ts_dict['kra5'], title='Model: KRA-5PeV')
fig.savefig('{}/ts_weighted_distribution_kra5.png'.format(plot_dir))

fig, axes = make_marginalized_plot(ts_dict['kra50'], title='Model: KRA-50PeV')
fig.savefig('{}/ts_weighted_distribution_kra50.png'.format(plot_dir))

fig, axes = make_marginalized_plot(ts_dict['pi0'], title='Model: $\pi^0$')
fig.savefig('{}/ts_weighted_distribution_pi0.png'.format(plot_dir))

#### Test Effect of Energy Range on fitted Normalization

In [ ]:
from tqdm.notebook import tqdm_notebook as tqdm


def run_fit_trials(n_trials, seed=42):
    fit_flux_dict = {k: [] for k in ['pi0', 'pi0_mixed', 'pi0_lowE']}
    fit_ns_dict = {k: [] for k in ['pi0', 'pi0_mixed', 'pi0_lowE']}
    fit_ts_dict = {k: [] for k in ['pi0', 'pi0_mixed', 'pi0_lowE']}
    fit_energy_dict = {k: [] for k in ['pi0', 'pi0_mixed', 'pi0_lowE']}
    fit_ts_total_dict = {k: [] for k in ['pi0', 'pi0_mixed', 'pi0_lowE']}
    
    for i in tqdm(range(n_trials), total=n_trials):
        seed_i = seed + i
        
        # create trial with our baseline analysis
        trial_pi0 = tr_dict['pi0'].get_one_trial(seed=seed_i, n_sig=ns_dict['pi0'], poisson=False)
        trial_pi0_mixed = tr_dict['pi0'].get_one_trial(seed=seed_i, n_sig=ns_dict['pi0_lowE'], poisson=False)

        # create trial for lowE analaysis to steal the signal events
        trial_pi0_lowE = tr_dict['pi0_lowE'].get_one_trial(seed=seed_i, n_sig=ns_dict['pi0_lowE'], poisson=False)

        # mix-and-match: combine signal from lowE to bkg trial of baseline analysis
        # This way we now have injected signal at lowE while lacking corresponding part at high E
        trial_pi0_mixed.evss[0][1] = trial_pi0_lowE.evss[0][1]

        # now we can perform a fit with both analyses. 
        # Both trials have the same signal injection, so difference in fitted normalization
        # may be caused by missing high-energy signal events in baseline analysis
        fit_dict = {
            'pi0': (tr_dict['pi0'], tr_dict['pi0'].get_one_fit_from_trial(trial_pi0, flat=False)),
            'pi0_mixed': (tr_dict['pi0'], tr_dict['pi0'].get_one_fit_from_trial(trial_pi0_mixed, flat=False)),
            'pi0_lowE': (tr_dict['pi0_lowE'], tr_dict['pi0_lowE'].get_one_fit_from_trial(trial_pi0_lowE, flat=False)),
        }
        for key, (tr, fit) in fit_dict.items():
            fit_flux_dict[key].append(tr.to_dNdE(fit[1]['ns']))
            fit_ns_dict[key].append(fit[1]['ns'])
            fit_ts_total_dict[key].append(fit[0])
        
        # get ts weights and energies 
        lr_pi0, mc_event_pi0 = compute_mc_lr(trial=trial_pi0, tr=tr_dict['pi0'], tr_trial=tr_dict['pi0'])
        lr_pi0_mixed, mc_event_pi0_mixed = compute_mc_lr(trial=trial_pi0_mixed, tr=tr_dict['pi0'], tr_trial=tr_dict['pi0_lowE'])
        lr_pi0_lowE, mc_event_pi0_lowE = compute_mc_lr(trial=trial_pi0_lowE, tr=tr_dict['pi0_lowE'], tr_trial=tr_dict['pi0_lowE'])
        
        fit_ts_dict['pi0'].append( 2 * np.log(lr_pi0))
        fit_ts_dict['pi0_mixed'].append( 2 * np.log(lr_pi0_mixed))
        fit_ts_dict['pi0_lowE'].append( 2 * np.log(lr_pi0_lowE))
        
        fit_energy_dict['pi0'].append(mc_event_pi0.true_energy)
        fit_energy_dict['pi0_mixed'].append(mc_event_pi0_mixed.true_energy)
        fit_energy_dict['pi0_lowE'].append(mc_event_pi0_lowE.true_energy)
        
    for key in fit_flux_dict.keys():
        fit_flux_dict[key] = np.array(fit_flux_dict[key])
        fit_ns_dict[key] = np.array(fit_ns_dict[key])
        fit_ts_total_dict[key] = np.array(fit_ts_total_dict[key])
        fit_ts_dict[key] = np.concatenate(fit_ts_dict[key])
        fit_energy_dict[key] = np.concatenate(fit_energy_dict[key])
    
    return fit_flux_dict, fit_ns_dict, fit_ts_total_dict, fit_ts_dict, fit_energy_dict

fit_flux_dict, fit_ns_dict, fit_ts_total_dict, fit_ts_dict, fit_energy_dict = run_fit_trials(100)


In [ ]:
for key in fit_flux_dict.keys():
    print('Test {}:'.format(key))
    print('  TS: {:5.1f} +- {:3.1f} | ns: {:6.1f} +- {:3.1f}'.format(
        np.mean(fit_ts_total_dict[key]), np.std(fit_ts_total_dict[key]),
        np.mean(fit_ns_dict[key]), np.std(fit_ns_dict[key]),
    ))
    print('  dNdE at 1 GeV: {:3.2e} +- {:3.2e}'.format(
        np.mean(fit_flux_dict[key]), np.std(fit_flux_dict[key]),
    ))
    weights = get_weights_from_ts(fit_ts_dict[key])
    for cl in [0.68, 0.9]:
        q_tail = (1. - cl) / 2.
        print('  TS-based Range {:2.0f}% CL: [{:3.1f} GeV, {:3.1f} GeV]'.format(
            cl*100.,
            weighted_quantile(fit_energy_dict[key], weights=weights, quantile=q_tail),
            weighted_quantile(fit_energy_dict[key], weights=weights, quantile=1. -q_tail),
        ))
        print('  True MC Range {:2.0f}% CL:  [{:3.1f} GeV, {:3.1f} GeV]'.format(
            cl*100.,
            weighted_quantile(fit_energy_dict[key], weights=None, quantile=q_tail),
            weighted_quantile(fit_energy_dict[key], weights=None, quantile=1. -q_tail),
        ))
    print()

#### "Unfold" true energy

Define mapping to go from: (delta_rec, E_rec) -> E_neutrino, i.e. P(E_neutrino | delta_rec, E_rec)

In [ ]:
import histlite as hl
from tqdm.notebook import tqdm_notebook as tqdm

def get_flux(tr):
    for key, item in tr.llh_kw['conf'].items():
        if 'flux' == key:
            return item
        if isinstance(item, dict):
            for key_sub, item_sub in item.items():
                if key_sub == 'flux':
                    return item_sub
    return None

   
def get_pdf(template_str, dec=a.bg_data.dec, energy=a.bg_data.energy, sigma=a.bg_data.sigma, 
            width_dec=np.deg2rad(45), width_log_energy=0.05, width_sigma=np.deg2rad(1000),
            ts_weighting_method='clip',
            disable_tqdm=False,
            ts=None, pdf_in_log=False, bins=None, cpus=15):
    """Get PDF in E_nu for given set of reconstructed values
    
    Notes: histlite's "contain()" method does not account for bin_width, i.e it assumes that 
    the value of particular bin is the probability of that bin. To achieve this, we must
    normalize the hist with the `density=True` flag, which will divide bin content by the
    bin width.
    """
    tr = tr_dict[template_str]
    if ts is None:
        ts = ts_dict[template_str]
    
    if bins is None:
        if pdf_in_log:
            bins = np.linspace(2, 7, 100)
        else:
            bins = np.logspace(2, 7, 100)
            
    # get ts-weights
    ts_weights = get_weights_from_ts(ts, method=ts_weighting_method)
    total_ts_weight = np.sum(ts_weights)
    
    assert len(ts_weights) == len(energy)
    assert len(ts_weights) == len(dec)
    assert len(ts_weights) == len(sigma)
    
    # assume flux of fitted model for weighting of MC    
    flux = get_flux(tr)
    weights = a.sig.oneweight * flux(a.sig.true_energy)

    log_energy = np.log10(energy)
       
    h_total = None
    hist_list = []
    sig_energy_weights = np.zeros_like(a.sig.true_energy)
    for dec_i, log_energy_i, sigma_i, w_i in tqdm(zip(dec, log_energy, sigma, ts_weights), total=len(dec), disable=disable_tqdm):
        mask_dec = np.logical_and(
            a.sig.dec > dec_i - width_dec,
            a.sig.dec < dec_i + width_dec,
        )
        mask_energy = np.logical_and(
            a.sig.log10energy > log_energy_i - width_log_energy,
            a.sig.log10energy < log_energy_i + width_log_energy,
        )
        mask_sigma = np.logical_and(
            a.sig.sigma > sigma_i - width_sigma,
            a.sig.sigma < sigma_i + width_sigma,
        )
        mask = np.logical_and(mask_dec, mask_energy)
        mask = np.logical_and(mask, mask_sigma)
        
        if np.sum(mask) < 500:
            print(np.sum(mask))
        
        # create PDF
        if pdf_in_log:
            data = np.log10(a.sig.true_energy[mask])
        else:
            data = a.sig.true_energy[mask]
        
        sig_energy_weights[mask] += (
            w_i * weights[mask] / np.sum(weights[mask]) / total_ts_weight
        )
        
        #hist = hl.hist(data, weights=weights[mask], bins=bins).normalize()
        hist = hl.hist(data, weights=weights[mask], bins=bins).normalize(density=True)
        #hist = hl.hist(data, weights=weights[mask], bins=bins) / np.diff(bins) / np.sum(weights[mask])
        hist_list.append(hist)
        
        if h_total is None:
            h_total = w_i / total_ts_weight * hist
        else:
            h_total += w_i / total_ts_weight * hist
    
    # print('SUM weights:', np.sum(sig_energy_weights))
    
    return h_total, hist_list, sig_energy_weights


if True:

    fig, ax = plt.subplots(figsize=(9, 6))
    color_cycler = get_color_cycler()
    #for dec in np.linspace(-np.pi/2., np.pi/2, 1):
    for dec in [0.]:
        for sigma in np.deg2rad([5]):
            for energy in np.logspace(np.log10(500), 6, 4):
            #for energy in np.logspace(3, 3, 1):
                color = next(color_cycler)
                h_total, hist_list, sig_energy_weights = get_pdf(
                    'pi0', dec=[dec], energy=[energy], sigma=[sigma], ts=[1.])
                hl.plot1d(h=h_total, ax=ax, color=color, label='{:3.1f} GeV | {:3.1f} ° | {:3.1f} °'.format(energy, np.rad2deg(dec), np.rad2deg(sigma)))
                ax.axvline(energy, color=color, ls='--')
                ax.axvline(weighted_quantile(a.sig.true_energy, weights=sig_energy_weights, quantile=0.5), color=color, ls='-')
                ax.axvline(h_total.contain(axis=0, frac=0.5).values, color=color, ls='-.')
    ax.legend()
    ax.set_xscale('log')
    ax.set_yscale('log')


#### Test Quantile Recovery

In [ ]:
def weighted_quantile2(values, quantiles, sample_weight=None, 
                      values_sorted=False, old_style=False):
    """ Very close to numpy.percentile, but supports weights.
    NOTE: quantiles should be in [0, 1]!
    :param values: numpy.array with data
    :param quantiles: array-like with many quantiles needed
    :param sample_weight: array-like of the same length as `array`
    :param values_sorted: bool, if True, then will avoid sorting of
        initial array
    :param old_style: if True, will correct output to be consistent
        with numpy.percentile.
    :return: numpy.array with computed quantiles.
    """
    values = np.array(values)
    quantiles = np.array(quantiles)
    if sample_weight is None:
        sample_weight = np.ones(len(values))
    sample_weight = np.array(sample_weight)
    assert np.all(quantiles >= 0) and np.all(quantiles <= 1), \
        'quantiles should be in [0, 1]'

    if not values_sorted:
        sorter = np.argsort(values)
        values = values[sorter]
        sample_weight = sample_weight[sorter]

    weighted_quantiles = np.cumsum(sample_weight) - 0.5 * sample_weight
    if old_style:
        # To be convenient with numpy.percentile
        weighted_quantiles -= weighted_quantiles[0]
        weighted_quantiles /= weighted_quantiles[-1]
    else:
        weighted_quantiles /= np.sum(sample_weight)
    return np.interp(quantiles, weighted_quantiles, values)

for q in np.linspace(0., 0.99, 10):
    print('q: {:3.2f}: {:8.1f} [True] | {:8.1f} [Hists] | {:8.1f} [Events]'.format(
        q, 
        np.quantile(a.sig.true_energy, q),
        weighted_quantile(a.sig.true_energy, quantile=q, weights=None),
        weighted_quantile2(a.sig.true_energy, quantiles=q, sample_weight=None),
    ))

In [ ]:
n = 100
fig, ax = plt.subplots()
ax.set_xscale('log')
ax.set_yscale('log')
h_total, hist_list, sig_energy_weights = get_pdf(
    'pi0', dec=a.sig.dec[:n], energy=a.sig.energy[:n], sigma=a.sig.sigma[:n], ts=np.ones_like(ts_dict['pi0'][:n]),
    #bins=np.logspace(1, 9, 50), width_log_energy=0.1,
)
hl.plot1d(h=h_total, ax=ax, label='h_total', ls=':')
h2 = hl.hist(a.sig.true_energy, weights=sig_energy_weights, bins=h_total.bins).normalize(density=True)
hl.plot1d(h=h2, ax=ax, label='h_events', ls='--')
ax.hist(a.sig.true_energy[:n], bins=h_total.bins[0], density=True, histtype='step', label='True')

for q in np.linspace(0., 0.99, 10):
    print('q: {:3.2f}: {:8.1f} [True] | {:8.1f} [Hists] | {:8.1f} [Events]'.format(
        q, 
        np.quantile(a.sig.true_energy[:n], q),
        #weighted_quantile(a.sig.true_energy[:n], quantile=q, weights=None),
        #weighted_quantile2(a.sig.true_energy[:n], quantiles=q, sample_weight=None),
        h_total.contain(0, frac=q).values,
        weighted_quantile(a.sig.true_energy, weights=sig_energy_weights, quantile=q),
    ))
ax.legend()

#### Compute Values

In [ ]:
%%time

from multiprocessing import Pool
import timeit

cpus = 3
ts_weighting_method = 'clip'  # GP paper uses 'clip'
#ts_weighting_method = 'all'

print('Starting pool with {} cpus'.format(cpus))

arg_list = ['pi0', 'kra5', 'kra50']
if cpus == 1:
    result = [get_pdf(k) for k in arg_list]
else:
    def get_pdf_multiprocessing(template_str):
        return get_pdf(template_str=template_str, ts_weighting_method=ts_weighting_method, disable_tqdm=True)
    
    with Pool(cpus) as p:
        start_t = timeit.default_timer()
        result = p.map(get_pdf_multiprocessing, arg_list)
        end_t = timeit.default_timer()
        print(end_t - start_t)

h_total_dict = {}
hist_list_dict = {}
sig_energy_weights_dict = {}
for i, key in enumerate(arg_list):
    h_total_dict[key] = result[i][0]
    hist_list_dict[key] = result[i][1]
    sig_energy_weights_dict[key] = result[i][2]
    

#### Compute Quantiles for Models [data derived energy range via TS]

The GP paper uses the data derived range via the positive TS-distribution of the experimental data events. 
Via the above generated PDFs, for every data event we can determine likely true neutrino energies for a given reconstructed declination and energy: $(\delta_\mathrm{rec}, E_\mathrm{rec}) \rightarrow E_\nu$, i.e. $P(E_\nu | \delta_\mathrm{rec}, E_\mathrm{rec})$. 
Essentially we are accumulating the weighted PDFs in true neutrino energies. The PDFs are weighted by the event's contribution to the overall analysis TS value. We use the "clipped" method, so events with TS contributions $<0$ are set to have weights equal to zero.

In [ ]:
for key in sig_energy_weights_dict.keys():
    print('Model {} [Data Derived TS]:'.format(key))
    for cl in [.68, 0.9]:
        q_tail = (1. - cl) / 2.
        print('  {:3.0f} GeV | {:3.1f}% CL: [{:3.0f}, {:3.0f}]'.format(
            np.average(a.sig.true_energy, weights=sig_energy_weights_dict[key]), cl*100.,
            weighted_quantile(a.sig.true_energy, weights=sig_energy_weights_dict[key], quantile=q_tail),
            weighted_quantile(a.sig.true_energy, weights=sig_energy_weights_dict[key], quantile=1-q_tail),
        ))


#### Formatting for table

In [ ]:
unit = 1000.
for key in sig_energy_weights_dict.keys():
    print('Model {} [Data Derived TS]:'.format(key))
    msg = '  '
    for q in [0.05, 0.16, 0.5, 0.84, 0.95]:
        msg += '{:3.2f} & '.format(
            weighted_quantile(a.sig.true_energy, weights=sig_energy_weights_dict[key], quantile=q) / unit)
    print(msg)


The remainder of this notebook computes alternative MC-based variations to determine the energy range of the analysis.

#### Compute MC expectation

Note that this is not exactly the same as the data-derived TS-based energy range, since the MC range here is only calculated on the injected signal events. To make this equal, the MC range here should be calculated based on the entire sample, i.e. background MC plus injected signal events

In [ ]:
for key, (lr, mc_events) in lr_mc_dict.items():
    
    weights = get_weights_from_ts(ts_mc_dict[key])
        
    print('Model {} [MC expectation]:'.format(key))
    for cl in [.68, 0.9]:
        q_tail = (1. - cl) / 2.
        print('  {:3.0f} GeV | {:3.1f}% CL: [{:3.0f}, {:3.0f}]'.format(
            np.average(mc_events.true_energy, weights=weights), cl*100.,
            weighted_quantile(mc_events.true_energy, weights=weights, quantile=q_tail),
            weighted_quantile(mc_events.true_energy, weights=weights, quantile=1-q_tail),
        ))


#### Compute MC expectation (neglects ts-weighting of MC distribution)

This is just the true distribution of injected signal MC

In [ ]:
for key in tr_dict.keys():
    
    flux = get_flux(tr_dict[key])
    weights = a.sig.oneweight * flux(a.sig.true_energy)
        
    print('Model {} [MC expectation neglecting TS-weights]:'.format(key))
    for cl in [.68, 0.9]:
        q_tail = (1. - cl) / 2.
        print('  {:3.0f} GeV | {:3.1f}% CL: [{:3.0f}, {:3.0f}]'.format(
            np.average(a.sig.true_energy, weights=weights), cl*100.,
            weighted_quantile(a.sig.true_energy, weights=weights, quantile=q_tail),
            weighted_quantile(a.sig.true_energy, weights=weights, quantile=1-q_tail),
        ))


#### Helper functions for Plots

In [ ]:
def get_injected_events(tr, n_sig, seed=None):
    injector = tr.sig_injs[0]
    original_keep = [k for k in injector.keep]
    if 'true_energy' not in injector.keep:
        injector.keep += ['true_energy']
    injected_events = injector.inject(n_sig, seed=seed)[0][0]
    injector.keep = original_keep
    return injected_events
    

In [ ]:

def plot_energy_dist(h_total, sig_energy_weights, cl=0.68, fig=None, ax=None, is_log=False, mc_tr=None, add_mc=False, mc_seed=42):
    if ax is None:
        fig, ax = plt.subplots(figsize=(9, 6))

    q_tail = (1. - cl) / 2.
    med_q = weighted_quantile(a.sig.true_energy, weights=sig_energy_weights, quantile=0.5)
    lower_q = weighted_quantile(a.sig.true_energy, weights=sig_energy_weights, quantile=q_tail)
    upper_q = weighted_quantile(a.sig.true_energy, weights=sig_energy_weights, quantile=1. - q_tail)
    
    if is_log:
        label = r'$E_\nu$: {:3.2f} GeV | {:.0f}% CL: [{:3.0f} GeV, {:3.0f} GeV])'.format(
            10**med_q, cl*100., 10**lower_q, 10**upper_q)
    else:
        label = r'$E_\nu$: {:3.2f} GeV | {:.0f}% CL: [{:3.0f} GeV, {:3.0f} GeV])'.format(
            med_q, cl*100., lower_q, upper_q)

    ax.axvline(lower_q, ls='--', color='0.7', label='{:.0f}% CL'.format(cl*100.))
    ax.axvline(upper_q, ls='--', color='0.7')
    ax.axvspan(lower_q, upper_q, color='0.7', alpha=.2)
    ax.axvline(med_q, ls='-', color='0.7', label='Median')

    hl.plot1d(ax=ax, h=h_total, label=label)
    #ax.hist(a.sig.true_energy, weights=sig_energy_weights, bins=h_total.bins[0], density=True, label='MPL')
    
    if mc_tr is not None and add_mc:
        if False:
            mc_events = get_injected_events(mc_tr, 1000000, seed=mc_seed)

            if is_log:
                h_mc = hl.hist(np.log10(mc_events.true_energy), bins=h_total.bins).normalize(density=True)
            else:
                h_mc = hl.hist(mc_events.true_energy, bins=h_total.bins).normalize(density=True)
            print(
                'MC:', 
                np.quantile(mc_events.true_energy, q_tail),
                np.quantile(mc_events.true_energy, 0.5),
                np.quantile(mc_events.true_energy, 1. - q_tail),
            )
            hl.plot1d(ax=ax, h=h_mc, label='MC [template sampled] (assumes uniform TS-weights)')

            flux = get_flux(mc_tr)
            weights = a.sig.oneweight * flux(a.sig.true_energy)
            h_mc_all = hl.hist(a.sig.true_energy, bins=h_total.bins, weights=weights).normalize(density=True)
            hl.plot1d(ax=ax, h=h_mc_all, label='MC [total MC] (assumes uniform TS-weights)')
        
        if True:
            lr, mc_events_ts = lr_mc_dict[key]
            weights = get_weights_from_ts(ts_mc_dict[key])
            print(
                'MC [injection trials]:', 
                weighted_quantile(mc_events_ts.true_energy, weights=weights, quantile=0.5),
                weighted_quantile(mc_events_ts.true_energy, weights=weights, quantile=q_tail),
                weighted_quantile(mc_events_ts.true_energy, weights=weights, quantile=1. - q_tail),
            )
            h_mc_ts = hl.hist(mc_events_ts.true_energy, bins=h_total.bins, weights=weights).normalize(density=True)
            hl.plot1d(ax=ax, h=h_mc_ts, label='MC [injection trials]')
        
        if False:
            if key + '_lowE' in lr_mc_dict:
                lr, mc_events_ts_lowE = lr_mc_dict[key + '_lowE']
                weights = get_weights_from_ts(ts_mc_dict[key + '_lowE'])
                h_mc_ts = hl.hist(mc_events_ts_lowE.true_energy, bins=h_total.bins, weights=weights).normalize(density=True)
                hl.plot1d(ax=ax, h=h_mc_ts, label='MC [injection trials] [lowE]')
            
    if is_log:
        ax.set_xlabel(r'$\log_{10}(E_{\nu} / \mathrm{GeV})$')
        ax.set_ylabel(r'$P \, ( \, \log_{10}[E_{\nu} \, / \, \mathrm{GeV}] \, )$')
    else:
        ax.set_xscale('log')
        ax.set_xlabel(r'$E_{\nu} / \mathrm{GeV}$')
        ax.set_ylabel(r'$P \, ( \, E_{\nu} \, / \, \mathrm{GeV} \, )$')
        
    ax.legend()
    return fig, ax


# Make Plot
for add_mc in [True, False]:
    fig, axes = plt.subplots(3, 1, figsize=(9, 9), sharex=True)
    
    for ax, key, name in zip(axes, ['pi0', 'kra5', 'kra50'], ['$\pi^0$', 'KRA-5PeV', 'KRA-50PeV']):
        plot_energy_dist(h_total=h_total_dict[key], sig_energy_weights=sig_energy_weights_dict[key], 
                         fig=fig, ax=ax, mc_tr=tr_dict[key], add_mc=add_mc)
        ax.set_title('Model: {}'.format(name))

    fig.tight_layout()
    
    if add_mc:
        for ax in axes:
            ax.set_yscale('log')
            ax.set_ylim(1e-8)
        fig.savefig('{}/ts_weighted_energy_with_mc.png'.format(plot_dir))
    else:
        fig.savefig('{}/ts_weighted_energy.png'.format(plot_dir))
        

## Compute MC-based TS Range

This is an approach to calculate an equivalent TS-based range for MC

In [ ]:
def get_mc_pseudo_trial(template_str, ns_dict=ns_dict, seed=None):
    trial = tr_dict[template_str].get_one_trial(n_sig=ns_dict[template_str],seed=seed)
    
    # move injected events to original data events to immitate
    # real data
    bkg_events = trial[0][0][0]
    sig_events = trial[0][0][1]
    
    # make keys match
    sig_events['log10energy'] = sig_events.log10energy
    
    # append signal evens
    events = cy.utils.Events.concatenate([bkg_events, sig_events])
    
    trial[0][0] = [events]
    
    return trial


In [ ]:
n_reps = 10

# get pseudo data
pseudo_trials = {}
for i in range(n_reps):
    pseudo_trials['pi0_{:02d}'.format(i)] = get_mc_pseudo_trial(template_str='pi0', seed=i)
    pseudo_trials['kra5_{:02d}'.format(i)] = get_mc_pseudo_trial(template_str='kra5', seed=i)
    pseudo_trials['kra50_{:02d}'.format(i)] = get_mc_pseudo_trial(template_str='kra50', seed=i)
    
lr_pseudo_dict = {}
for i in range(n_reps):
    for k in ['pi0', 'kra5', 'kra50']:
        key = '{}_{:02d}'.format(k, i)
        lr_pseudo_dict[key] = get_lr_from_trial(trial=pseudo_trials[key], tr=tr_dict[k])

ts_pseudo_dict = {}
for key, lr in lr_pseudo_dict.items():
    ts = 2 * np.log(lr)
    print(key, np.sum(ts))
    ts_pseudo_dict[key] = ts


In [ ]:
cpus = 5 #3 * n_reps

print('Starting pool with {} cpus'.format(cpus))

# define method to compute PDF for these pseudo trials
def get_pdf_from_mc_pseudo_data(key):
    template_str = key.split('_')[0]
    pseudo_data = pseudo_trials[key][0][0][0]

    return get_pdf(
        template_str=template_str, 
        dec=pseudo_data.dec, 
        energy=pseudo_data.energy, 
        sigma=pseudo_data.sigma, 
        ts=ts_pseudo_dict[key],
        ts_weighting_method=ts_weighting_method,
        disable_tqdm=True,
    )

arg_list = list(ts_pseudo_dict.keys())
if cpus == 1:
    result = [get_pdf_from_mc_pseudo_data(k) for k in arg_list]
else:
    with Pool(cpus) as p:
        start_t = timeit.default_timer()
        result = p.map(get_pdf_from_mc_pseudo_data, arg_list)
        end_t = timeit.default_timer()
        print(end_t - start_t)

h_total_dict_pseudo = {}
hist_list_dict_pseudo = {}
sig_energy_weights_dict_pseudo = {}
for i, key in enumerate(arg_list):
    h_total_dict_pseudo[key] = result[i][0]
    hist_list_dict_pseudo[key] = result[i][1]
    sig_energy_weights_dict_pseudo[key] = result[i][2]
    

##### Compute Quantiles for MC TS (without caveats)

In [ ]:
for key in ['pi0', 'kra5', 'kra50']:
    print('Model {} [MC ts-equivalent]:'.format(key))
    for cl in [.68, 0.9]:
        q_tail = (1. - cl) / 2.
        
        averages = []
        lower_bounds = []
        upper_bounds = []
        for k in sig_energy_weights_dict_pseudo.keys():
            if key == k.split('_')[0]:
                averages.append(np.average(a.sig.true_energy, weights=sig_energy_weights_dict_pseudo[k]))
                lower_bounds.append(weighted_quantile(a.sig.true_energy, weights=sig_energy_weights_dict_pseudo[k], quantile=q_tail))
                upper_bounds.append(weighted_quantile(a.sig.true_energy, weights=sig_energy_weights_dict_pseudo[k], quantile=1-q_tail))
        print('  {:3.0f} +- {:3.0f} GeV | {:3.1f}% CL: [{:3.0f} +- {:3.0f}, {:3.0f} +- {:3.0f}]'.format(
            np.median(averages), np.std(averages, ddof=1), cl*100.,
            np.median(lower_bounds), np.std(lower_bounds, ddof=1),
            np.median(upper_bounds), np.std(upper_bounds, ddof=1),
        ))


##### Formatting for Table

In [ ]:
unit = 1000.
for key in ['pi0', 'kra5', 'kra50']:
    print('Model {} [MC ts-equivalent]:'.format(key))
    msg = '  '
    for q in [0.05, 0.16, 0.5, 0.84, 0.95]:
        
        values = []
        for k in sig_energy_weights_dict_pseudo.keys():
            if key == k.split('_')[0]:
                values.append(weighted_quantile(a.sig.true_energy, weights=sig_energy_weights_dict_pseudo[k], quantile=q))
        
        values = np.array(values) / unit
        
        std = np.std(values, ddof=1)
        if std > 0.1:
            msg += '${:3.1f} \pm {:3.1f}$ & '.format(np.median(values), std)
        elif std > 0.01:
            msg += '${:3.2f} \pm {:3.2f}$ & '.format(np.median(values), std)
        else:
            msg += '${:3.3f} \pm {:3.3f}$ & '.format(np.median(values), std)
    print(msg)


In [ ]:
for key in sig_energy_weights_dict_pseudo.keys():
    print('Model {} [MC ts-equivalent]:'.format(key))
    for cl in [.68, 0.9]:
        q_tail = (1. - cl) / 2.
        print('  {:3.0f} GeV | {:3.1f}% CL: [{:3.0f}, {:3.0f}]'.format(
            np.average(a.sig.true_energy, weights=sig_energy_weights_dict_pseudo[key]), cl*100.,
            weighted_quantile(a.sig.true_energy, weights=sig_energy_weights_dict_pseudo[key], quantile=q_tail),
            weighted_quantile(a.sig.true_energy, weights=sig_energy_weights_dict_pseudo[key], quantile=1-q_tail),
        ))
